# How much is your Fiat 500 worth?
## Skills: Python, Web scraping, Geopy, Regression

Download the Jupyter Notebook from [GitHub](https://nicomoriuala.github.io)

<a id='content' />

## Contents

1. [Introduction](#intro)
1. [Web Scraping](#scraping)
2. [Data Cleansing](#cleansing)
3. [Mapping](#mapping)
4. [A definir](#volatility)

<a id='intro' />

## Introduction
[(back to top)](#content)

In [2]:
#Importing packages
import pandas as pd
import numpy as np
import re
import unidecode

from tqdm.notebook import tqdm
import time

from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException

<a id='scraping' />

## Web Scraping
[(back to top)](#content)

In [36]:
#Load webdriver
driver = webdriver.Chrome('/Users/nicolamori/Library/Application Support/binman_chromedriver/mac64/85.0.4183.38/chromedriver')

#Open page
driver.get('https://www.automobile.it/fiat-500-usate?anno_fino_a=2020&immatricolazione=2007')
time.sleep(4)

#Close cookies popup
driver.find_element_by_xpath('//*[@id="onetrust-accept-btn-handler"]').click()
time.sleep(2)

In [37]:
#Empty list and df
temp = []
df_ = pd.DataFrame(columns=['Title', 'Province','Condition', 'Month_Year', 'Kms', 'Fuel', 'Transmission', 'Price'])

#Number of pages
npages = int(driver.find_element_by_xpath('//*[@id="__next"]/main/div[2]/div/div[4]/div[2]/div[3]/div[1]/button[4]').text)

for h in tqdm(range(npages), position = 0, desc='1st loop'):
    

    #Title and Province
    titles=driver.find_elements_by_xpath('//*[@class="jsx-3257049061 jsx-2759044879"]')
    for title in titles: temp.append(title.text)
    
    del temp[1::3]
    temp = [temp[x:x+2] for x in range(0, len(temp),2)]
    
    #Dataframe and clear list
    part_1 = pd.DataFrame(temp, columns=['Title', 'Province'])  
    temp.clear()
    
    
    #Specs
    specs=driver.find_elements_by_xpath('//*[@class="jsx-3257049061 jsx-2759044879 Card__InfoTag font-base"]')
    for spec in specs: temp.append(spec.text)
    
    while 'Unico proprietario' in temp: temp.remove('Unico proprietario')
    while '2 Proprietari' in temp: temp.remove('2 Proprietari')
    while '3 Proprietari' in temp: temp.remove('3 Proprietari')
    while '4 Proprietari' in temp: temp.remove('4 Proprietari')
        
    size = len(temp) 
    idx_list = [idx for idx, val in enumerate(temp) if val == 'Usato'] 
    temp = [temp[i: j] for i, j in zip([0] + idx_list, idx_list + ([size] if idx_list[-1] != size else []))] 
    del temp[0:0 + 1]
    
    #Dataframe and clear list
    part_2 = pd.DataFrame(temp, columns=['Condition', 'Month_Year', 'Kms', 'Fuel', 'Transmission'])
    temp.clear()
    
    
    #Price
    prices=driver.find_elements_by_xpath('//*[@class="jsx-3257049061 jsx-2759044879 Card__InfoPrice font-medium"]')
    for price in prices: temp.append(price.text)
        
    #Dataframe and clear list
    part_3 = pd.DataFrame(temp, columns=['Price'])
    temp.clear()
    
    
    #Combine parts and append to dataframe
    df = pd.concat([part_1.reset_index(drop=True), part_2.reset_index(drop=True), part_3.reset_index(drop=True)], axis=1)
    df_ = df_.append(df, ignore_index = True)
    
    
    #Switch to next page or close
    if h < max(range(npages)):
        driver.find_element_by_xpath('//*[@class="jsx-3190051910 font-base xxsmall primary-orange styled"]').click()
        time.sleep(3)
    else: driver.close()
    
    
#df_.to_csv('fiat500_prices.tsv', sep='\t', index=False)


CPU times: user 26.1 s, sys: 2.32 s, total: 28.4 s
Wall time: 20min 55s


In [42]:
fiat_500=pd.read_csv('fiat500_prices.tsv', sep='\t')

del fiat_500['Owners']
del fiat_500['Condition']

fiat_500 = fiat_500[~fiat_500['Title'].str.contains('Abarth', case=False)]
fiat_500 = fiat_500[~fiat_500['Title'].str.contains('C ', case=False)]

fiat_500.info()

fiat_500.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2656 entries, 1 to 3148
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Title         2656 non-null   object
 1   Province      2656 non-null   object
 2   Month_Year    2656 non-null   object
 3   Kms           2656 non-null   object
 4   Fuel          2656 non-null   object
 5   Transmission  2646 non-null   object
 6   Price         2656 non-null   object
dtypes: object(7)
memory usage: 166.0+ KB


,Title,Province,Month_Year,Kms,Fuel,Transmission,Price
1,FIAT 500,Giugliano in Campania (NA),Luglio 2013,51.000 km,Benzina - Euro 5,Manuale,€ 4.990
2,Fiat 500 1.2 69cv Collezione Tetto,Ciampino (RM),Gennaio 2019,31.412 km,Benzina - Euro 6,Manuale,€ 9.980
3,FIAT 500 1.2 Lounge,Moncalieri (TO),Aprile 2019,14.918 km,Benzina - Euro 6,Manuale,€ 9.900
4,FIAT 500 1.2 Lounge,Torino (TO),Maggio 2019,18.539 km,Benzina - Euro 6,Manuale,€ 9.700
5,Fiat 500 1.2 69 Cv Lounge Navi,Roma (RM),Marzo 2019,21.445 km,Benzina,Manuale,€ 9.780


In [43]:
fiat_500['Kms'] = ([i.replace(' km','') for i in fiat_500['Kms']])
fiat_500['Kms'] = pd.to_numeric(fiat_500['Kms'])

fiat_500['Price'] = ([i.replace('€ ','') for i in fiat_500['Price']])
fiat_500['Price'] = pd.to_numeric(fiat_500['Price'])

fiat_500['Fuel'] = ([i.replace(' - Euro ','') for i in fiat_500['Fuel']])
fiat_500['Fuel'] = fiat_500['Fuel'].str.replace('\d+', '')

fiat_500.info()
fiat_500.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2656 entries, 1 to 3148
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Title         2656 non-null   object 
 1   Province      2656 non-null   object 
 2   Month_Year    2656 non-null   object 
 3   Kms           2656 non-null   float64
 4   Fuel          2656 non-null   object 
 5   Transmission  2646 non-null   object 
 6   Price         2656 non-null   float64
dtypes: float64(2), object(5)
memory usage: 166.0+ KB


,Title,Province,Month_Year,Kms,Fuel,Transmission,Price
1,FIAT 500,Giugliano in Campania (NA),Luglio 2013,51.000,Benzina,Manuale,4.99
2,Fiat 500 1.2 69cv Collezione Tetto,Ciampino (RM),Gennaio 2019,31.412,Benzina,Manuale,9.98
3,FIAT 500 1.2 Lounge,Moncalieri (TO),Aprile 2019,14.918,Benzina,Manuale,9.90
4,FIAT 500 1.2 Lounge,Torino (TO),Maggio 2019,18.539,Benzina,Manuale,9.70
5,Fiat 500 1.2 69 Cv Lounge Navi,Roma (RM),Marzo 2019,21.445,Benzina,Manuale,9.78


In [45]:
conditions = [fiat_500['Title'].str.contains('Lounge', case=False), 
              fiat_500['Title'].str.contains('Pop', case=False), 
              fiat_500['Title'].str.contains('Sport', case=False), 
              fiat_500['Title'].str.contains('Collezione', case=False), 
              fiat_500['Title'].str.contains('Star', case=False), 
              fiat_500['Title'].str.contains('S ', case=False), 
              fiat_500['Title'].str.contains(' S', case=False),  
              fiat_500['Title'].str.contains('"S"', case=False)]

choices = ['Lounge', 'Pop', 'Sport', 'Collezione', 'Star', 'Sport', 'Sport', 'Sport']

fiat_500['Version'] = np.select(conditions, choices, default='None')

fiat_500['Version']

fiat_500.Version.value_counts()

Lounge        1685
Pop            396
Sport          323
None           167
Star            62
Collezione      23
Name: Version, dtype: int64

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2656 entries, 1 to 3148
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Title         2656 non-null   object 
 1   Province      2656 non-null   object 
 2   Month_Year    2656 non-null   object 
 3   Kms           2656 non-null   float64
 4   Fuel          2656 non-null   object 
 5   Transmission  2646 non-null   object 
 6   Price         2656 non-null   float64
 7   Version       2656 non-null   object 
dtypes: float64(2), object(6)
memory usage: 186.8+ KB


In [20]:
#driver.close()